In [ ]:
# import all packages to use in this project

import os
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import glob
import natsort
import shutil
from glom import glom

In [ ]:
from glass.sql.db import create_pgdb
from glass.sql.tbl import create_tbl
from glass.dtt.mge.pd import merge_df
from glass.rd.shp import shp_to_obj
from glass.prop.sql import lst_tbl
from glass.wt.sql import df_to_db

In [ ]:
###### Variaveis a serem alteradas ######

stations = '/home/yuri/fireloc-system/firerisk/data/stations.geojson'
observations = '/home/yuri/fireloc-system/firerisk/data/'
portugal = '/home/yuri/fireloc-system/firerisk/data/caop_pt.shp'
db = 'ipma2'

#########################################

In [ ]:
datab = create_pgdb(db)

In [ ]:
tbl_varia = {
    'variavel' : {'id' : 'int', 'nome' : 'varchar(30)'}
}

In [ ]:
tbl_sta = {'estacao': {
    'id' : 'int', 'idEstacao' : 'int', 'localEstacao' : 'varchar(80)',
    'geom' : 'geometry'
}}

In [ ]:
tbl_values = {
    'valor' : {'id' : 'int', 'id_est' : 'int', 'id_var' : 'int', 'daytime' : 'varchar(20)', 'valor' : 'float'}
}

In [ ]:
tbl_v = create_tbl(db, tbl_varia)

In [ ]:
tbl_e = create_tbl(db, tbl_sta)

In [ ]:
tbl_vl = create_tbl(db, tbl_values)

In [ ]:
pt = shp_to_obj(portugal)

In [ ]:
pt_dis = pt.unary_union

In [ ]:
pt_diss = gpd.GeoDataFrame(geometry=[pt_dis], crs = 'EPSG:3763')

In [ ]:
sdata = json.load(open(stations, 'r'))

In [ ]:
gjson = {"type": "FeatureCollection", "features" : sdata}

In [ ]:
sdf= gpd.GeoDataFrame.from_features(gjson, crs="EPSG:4326").to_crs(3763)
#sdf_crs = sdf.to_crs(3763)

In [ ]:
sdf_pt = sdf.clip(pt_diss)

In [ ]:
sdf_pt.index = np.arange(1, len(sdf_pt) + 1)

In [ ]:
sdf_pt['id'] = sdf_pt.index

In [ ]:
variaveis = [
    'intensidadeVentoKM', 'temperatura', 'radiacao', 'idDireccVento',
    'precAcumulada', 'intensidadeVento', 'humidade', 'pressao'
]

In [ ]:
varlst = [[i + 1, variaveis[i]] for i in range(len(variaveis))]

variaveis = pd.DataFrame(varlst, columns=['id', 'nome'])

In [ ]:
obser = glob.glob(observations + '**.json')

In [ ]:
for o in obser:
    odata = json.load(open(o, 'r'))
    
    obs = []
    
    for d in odata:
        for station in odata[d]:
            if not odata[d][station]: continue

            odata[d][station]['daytime']= d
            odata[d][station]['idEstacao']= station

            obs.append(odata[d][station])

    df=pd.DataFrame(obs)
    
    cols = variaveis.nome.tolist()

    dfs = []

    df_cols = df.columns.values
    icols = ['idEstacao', 'daytime']
    for col in cols:
        _df = df.copy(deep=True)

        _df.drop([c for c in df_cols if c != col and c not in icols], axis=1, inplace=True)

        _df.rename(columns={col: 'valor'}, inplace=True)

        _df['varid'] =col

        dfs.append(_df)
        
    _all = merge_df(dfs)
        
    _all['id_var'] = 0
    
    for i, r in variaveis.iterrows():
        _all['id_var'] = np.where(
            _all.varid == r['nome'],
            r['id'], _all['id_var']
        )
    
    _all['idEstacao'] = pd.to_numeric(_all['idEstacao'])
    
    sdf_pt['idEstacao'] = pd.to_numeric(sdf_pt['idEstacao'])
    
    _all['id_est'] = 0

    for i, r in sdf_pt.iterrows():

        _all['id_est'] = np.where(
            _all.idEstacao == r['idEstacao'],
            r['id'], _all['id_est']
        )
    
    fdf = _all.loc[(_all["id_est"] > 0)]
    
    fdf.drop('varid', axis=1, inplace=True)
    fdf.drop('idEstacao', axis=1, inplace=True)
    
    fdf.index = np.arange(1, len(fdf) + 1)
    fdf['id'] = fdf.index
    
    variaveis = variaveis[['id', 'nome']]
    sdf_pt = sdf_pt[['id', 'idEstacao', 'localEstacao', 'geometry']]
    fdf = fdf[['id', 'id_var', 'id_est', 'daytime', 'valor']]
    
    df_db_v  = df_to_db(db, variaveis, "variavel")
    df_db_st = df_to_db(
        db, sdf_pt, "estacao",
        epsg=3763, geom_type="POINT",
        col_geom="geometry"
    )
    df_db_vl = df_to_db(db, fdf, "valor")
    print(fdf)

In [ ]:
fdf